<a href="https://colab.research.google.com/github/Aviva-Chen/bertForClassification_/blob/main/bertSequenceCla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
path='/content/gdrive/MyDrive/bertFolder'
os.chdir(path)
print(os.getcwd())

Mounted at /content/gdrive
/content/gdrive/MyDrive/bertFolder


In [ ]:
!pip3 install torchvision
!pip install transformers

In [ ]:
import pandas as pd
train_data=[]
labels=[]

df = pd.read_excel("/content/gdrive/MyDrive/bertFolder/wholeDataset.xlsx", names=['title', 'label'])
print('finish loading')

from sklearn.utils import shuffle
df1 = shuffle(df)


train_data0=list(df1['title'])

labels=list(df1['label'])
for title in train_data0:
    train_data.append(str(title))
 

finish loading


In [ ]:
import pandas as pd
import torch
from sklearn.metrics import accuracy_score
from torch import nn, optim
from transformers import BertModel, BertTokenizer,DistilBertTokenizer, BertForSequenceClassification, BertConfig
#将数据转化为data_loader形式
tokenizer=DistilBertTokenizer.from_pretrained('distilbert-base-uncased')  

max_length=32
traindata_tokened = tokenizer(train_data,padding=True,truncation=True,max_length=max_length,return_tensors='pt')
labels=torch.tensor(labels)

#encoding data
from torch.utils.data import TensorDataset,Dataset,DataLoader,random_split, RandomSampler, SequentialSampler
class DataToDataset(Dataset):
    def __new__(cls,encoding,labels):
        return Dataset.__new__(cls)
    def __init__(self,encoding,labels):
        self.encoding=encoding
        self.labels=labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self,index):
        return self.encoding['input_ids'][index],self.encoding['attention_mask'][index],self.labels[index]
train_dataset = DataToDataset(traindata_tokened,labels)

#封装数据,labels是训练数据的标签，targets是测试数据的标签
train_size = int(0.95 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])


BATCH_SIZE=64
train_loader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,shuffle=True)
val_loader = DataLoader(val_dataset,batch_size = BATCH_SIZE,shuffle=True)

In [ ]:
#建立模型
from transformers import DistilBertForSequenceClassification, AdamW, BertConfig

#model = BertForSequenceClassification.from_pretrained("Bert-base-uncased",num_labels = 2, output_attentions = False,output_hidden_states = False)
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", output_attentions = False,output_hidden_states = False)

# Tell pytorch to run this model on the GPU.
model.cuda()

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    
    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [ ]:
import random
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

optimizer = AdamW(model.parameters(),lr = 1e-5,eps = 1e-8)

from transformers import get_linear_schedule_with_warmup
epochs=3
total_steps = len(train_loader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps = 0,num_training_steps = total_steps)

training_stats=[]
# Store our loss and accuracy for plotting
train_loss_set = []
total_t0=time.time()

for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()
    total_train_loss = 0

    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_loader):

        if step % 500 == 0 and not step == 0:
          elapsed = format_time(time.time() - t0)
          print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_loader), elapsed))
        
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()

        #outputs= model(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)
        outputs= model(b_input_ids,attention_mask=b_input_mask,labels=b_labels)
    
        #print(outputs)
        loss=outputs[0]
        logits=outputs[1]
       
        train_loss_set.append(loss.item())
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_loader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in val_loader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            outputs = model(b_input_ids,attention_mask=b_input_mask,labels=b_labels)
            #outputs = model(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)
            loss=outputs[0]
            logits=outputs[1]
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(val_loader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(val_loader)

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("----")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time() - total_t0)))



In [ ]:
import os
os.chdir('/content/gdrive/MyDrive/bertFolder')

#保存模型
torch.save(model,'/content/gdrive/MyDrive/bertFolder/model2.pt')


In [ ]:
#总结训练过程
pd.set_option('precision',2)
df_stats=pd.DataFrame(data=training_stats)
df_stats=df_stats.set_index('epoch')
print(df_stats)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()

plt.figure(figsize=(15,8))
plt.title("Training loss")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.plot(train_loss_set)
plt.show()


In [ ]:
#测试集性能
import pandas as pd

test_df=pd.read_excel('/content/gdrive/MyDrive/bertFolder/testSet.xlsx',sheet_name='Sheet3',names=['title given by machine','label'])
test_data0=list(test_df['title given by machine'])

sentences=[str(i) for i in test_data0]
labels=list(test_df['label'])

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    print(sent)
    encoded_dict = tokenizer.encode_plus(
        sent,  # Sentence to encode.
        add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
        max_length=64,  # Pad & truncate all sentences.
        pad_to_max_length=True,
        return_attention_mask=True,  # Construct attn. masks.
        return_tensors='pt',  # Return pytorch tensors.
    )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Set the batch size.
batch_size = 32

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

model.eval()

predictions , true_labels = [], []

total_test_accuracy=0

for batch in prediction_dataloader:
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_labels = batch

  with torch.no_grad():
    outputs = model(b_input_ids, token_type_ids=None,attention_mask=b_input_mask)
    logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()

  predictions.append(logits)
  true_labels.append(label_ids)

  total_test_accuracy += flat_accuracy(logits, label_ids)

  avg_test_accuracy = total_test_accuracy / len(prediction_dataloader)
  print("  Accuracy: {0:.3f}".format(avg_test_accuracy))


print('DONE.')
print('Positive samples: %d of %d (%.2f%%)'%(labels.sum(),len(labels),(labels.sum()/len(labels)*100.0)))

from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import classification_report

matthews_set = []

# Evaluate each test batch using Matthew's correlation coefficient
print('Calculating Matthews Corr. Coef. for each batch...')

# For each input batch...
for i in range(len(true_labels)):
    pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
    matthews = matthews_corrcoef(true_labels[i], pred_labels_i)
    matthews_set.append(matthews)

ax = sns.barplot(x=list(range(len(matthews_set))), y=matthews_set, ci=None)

plt.title('MCC Score per Batch')
plt.ylabel('MCC Score (-1 to +1)')
plt.xlabel('Batch #')

plt.show()

# Combine the results across all batches.
flat_predictions = np.concatenate(predictions, axis=0)

# For each sample, pick the label (0 or 1) with the higher score.
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = np.concatenate(true_labels, axis=0)

print(classification_report(flat_predictions,flat_true_labels,digits=3))

# Calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

print('Total MCC: %.3f' % mcc)

In [1]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/Aviva-Chen/bertForClassification.git

Cloning into 'bertForClassification'...
